In [1]:
!pip install lightgbm
!pip install joblib
!pip install dask
!pip install catboost

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 8.0 MB/s eta 0:00:00


In [2]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import lightgbm as lgb
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import joblib
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, classification_report
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, HistGradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [3]:
# Cargar los datos
train_data = pd.read_csv("kaggle_train_beta.csv")
test_data = pd.read_csv("kaggle_test_beta.csv")
submission_example = pd.read_csv("submission_example_beta.csv")

In [4]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 275132 entries, 0 to 275131
Data columns (total 17 columns):
 #   Column                        Non-Null Count   Dtype 
---  ------                        --------------   ----- 
 0   CÓDIGO                        275132 non-null  int64 
 1   PERIODO_RESULTADOS            275132 non-null  int64 
 2   CARRERA_PROFESIONAL           275132 non-null  object
 3   CIUDAD                        275132 non-null  object
 4   COSTO_MATRÍCULA_SOLES         272893 non-null  object
 5   HORAS_SEMANALES_TRABAJO       263595 non-null  object
 6   NIVEL_SOCIECONÓMICO_VIVIENDA  263239 non-null  object
 7   CUENTA_CON_INTERNET           265366 non-null  object
 8   NIVEL_EDUCATIVO_PADRE         266830 non-null  object
 9   CUENTA_CON_LAVADORA           260407 non-null  object
 10  TIENE_CARRO                   258962 non-null  object
 11  ANTECEDENTES_PENALES          275132 non-null  object
 12  PAGÓ_MATRÍCULA_POR_SU_CUENTA  272801 non-null  object
 13 

In [5]:
train_data.head()

,CÓDIGO,PERIODO_RESULTADOS,CARRERA_PROFESIONAL,CIUDAD,COSTO_MATRÍCULA_SOLES,HORAS_SEMANALES_TRABAJO,NIVEL_SOCIECONÓMICO_VIVIENDA,CUENTA_CON_INTERNET,NIVEL_EDUCATIVO_PADRE,CUENTA_CON_LAVADORA,TIENE_CARRO,ANTECEDENTES_PENALES,PAGÓ_MATRÍCULA_POR_SU_CUENTA,CUENTA_CON_ORDENADOR,CUENTA_CON_INTERNET.1,NIVEL_EDUCATIVO_MADRE,RENDIMIENTO_ACADÉMICO
0,133959,20203,ADMINISTRACIÓN EN SALUD OCUPACIONAL,LIMA,Entre 850 y menos de 2000 soles,Entre 11 y 20 horas,Nivel D,No,Primaria incompleta,Si,Si,N,Si,No,No,Primaria incompleta,medio-bajo
1,784510,20212,ADMINISTRACION DE EMPRESAS,LIMA,Entre 850 y menos de 2000 soles,Más de 30 horas,Nivel C,Si,Primaria incompleta,Si,No,N,Si,Si,Si,Primaria incompleta,medio-alto
2,535024,20183,INGENIERIA AMBIENTAL Y SANITARIA,CAJAMARCA,Entre 850 y menos de 2000 soles,Menos de 10 horas,Nivel D,No,Primaria completa,Si,No,N,No,Si,No,Primaria completa,medio-alto
3,591650,20212,CONTADURIA PUBLICA,LIMA,Entre 850 y menos de 2000 soles,Más de 30 horas,Nivel C,Si,Primaria incompleta,Si,No,N,No,Si,Si,Primaria completa,medio-bajo
4,635933,20195,CONTADURIA PUBLICA,PIURA,Entre 2000 y menos de 3500 soles,Más de 30 horas,Nivel C,Si,Educación profesional completa,Si,No,N,No,Si,Si,Secundaria (Bachillerato) incompleta,medio-alto


In [6]:
train_data.shape

(275132, 17)

In [7]:
 # Verificamo las columnas por si tienen valores nulos
print(train_data.isnull().sum())

CÓDIGO                              0
PERIODO_RESULTADOS                  0
CARRERA_PROFESIONAL                 0
CIUDAD                              0
COSTO_MATRÍCULA_SOLES            2239
HORAS_SEMANALES_TRABAJO         11537
NIVEL_SOCIECONÓMICO_VIVIENDA    11893
CUENTA_CON_INTERNET              9766
NIVEL_EDUCATIVO_PADRE            8302
CUENTA_CON_LAVADORA             14725
TIENE_CARRO                     16170
ANTECEDENTES_PENALES                0
PAGÓ_MATRÍCULA_POR_SU_CUENTA     2331
CUENTA_CON_ORDENADOR            14032
CUENTA_CON_INTERNET.1            9766
NIVEL_EDUCATIVO_MADRE            8469
RENDIMIENTO_ACADÉMICO               0
dtype: int64


In [8]:
# Eliminar columnas redundantes
train_data = train_data.drop(columns=['CUENTA_CON_INTERNET.1'])
test_data = test_data.drop(columns=['CUENTA_CON_INTERNET.1'])

In [9]:
# Separar características y etiqueta objetivo
target_col = 'RENDIMIENTO_ACADÉMICO'
X = train_data.drop(columns=[target_col, 'CÓDIGO'])
y = train_data[target_col]

In [10]:
# Codificar la etiqueta objetivo
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [11]:
# Dividir en conjuntos de entrenamiento y validación
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

In [12]:
# Identificar columnas categóricas y numéricas
categorical_cols = X.select_dtypes(include=['object']).columns
numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns

In [13]:
# Imputadores para manejar valores faltantes
categorical_imputer = SimpleImputer(strategy='most_frequent')
numerical_imputer = SimpleImputer(strategy='mean')

In [14]:
# Pipelines para preprocesamiento
categorical_pipeline = Pipeline(steps=[
    ('imputer', categorical_imputer),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
numerical_pipeline = Pipeline(steps=[
    ('imputer', numerical_imputer),
    ('scaler', StandardScaler())
])

In [15]:
# Combinación de preprocesamiento
preprocessor = ColumnTransformer(transformers=[
    ('num', numerical_pipeline, numerical_cols),
    ('cat', categorical_pipeline, categorical_cols)
])

In [16]:
# Preprocesar conjuntos de datos
X_train_preprocessed = preprocessor.fit_transform(X_train)
X_val_preprocessed = preprocessor.transform(X_val)
X_test_preprocessed = preprocessor.transform(test_data.drop(columns=['CÓDIGO']))

In [17]:
models = {
    #
    # 'Random Forest': RandomForestClassifier(n_estimators=300, max_depth=25, random_state=42),
    # 'Gradient Boosting': GradientBoostingClassifier(n_estimators=300, learning_rate=0.1, max_depth=5, random_state=42),
    'HistGradient Boosting': HistGradientBoostingClassifier(max_iter=250, max_depth=10, random_state=42),

    # Modelos de árboles de decisión avanzados
    # 'XGBoost': xgb.XGBClassifier(n_estimators=300, max_depth=15, random_state=42, learning_rate=0.05, use_label_encoder=False, eval_metric='logloss'),
    # 'CatBoost': cb.CatBoostClassifier(iterations=300, depth=10, learning_rate=0.05, verbose=0, random_seed=42),

    # Métodos lineales
    # 'Logistic Regression': LogisticRegression(max_iter=3000, random_state=42),
}


In [18]:
  # Entrenar y evaluar cada modelo
best_model = None
best_accuracy = 0
for name, model in models.items():
    print(f"Entrenando modelo: {name}")
    model.fit(X_train_preprocessed, y_train)
    y_val_pred = model.predict(X_val_preprocessed)
    acc = accuracy_score(y_val, y_val_pred)
    print(f"{name} Accuracy: {acc}")
    print(classification_report(y_val, y_val_pred, target_names=label_encoder.classes_))
    if acc > best_accuracy:
        best_accuracy = acc
        best_model = model

Entrenando modelo: HistGradient Boosting
HistGradient Boosting Accuracy: 0.5985788794591745
              precision    recall  f1-score   support

  medio-alto       0.60      0.58      0.59     27472
  medio-bajo       0.60      0.62      0.61     27555

    accuracy                           0.60     55027
   macro avg       0.60      0.60      0.60     55027
weighted avg       0.60      0.60      0.60     55027



In [19]:
print(best_model)

HistGradientBoostingClassifier(max_depth=10, max_iter=250, random_state=42)


In [20]:
# Guardar el mejor modelo
joblib.dump(best_model, 'Chisma.pkl')

['Chisma.pkl']

In [21]:
# Predicciones en el conjunto de prueba
X_test = test_data.drop(columns=['CÓDIGO'])
X_test_preprocessed = preprocessor.transform(X_test)
test_codes = test_data['CÓDIGO']
y_test_pred = best_model.predict(X_test_preprocessed)

In [22]:
# Convertir predicciones a etiquetas originales
y_test_pred_labels = label_encoder.inverse_transform(y_test_pred)

In [24]:
# Crear archivo de predicción
submission = pd.DataFrame({
    'CÓDIGO': test_codes,
    'RENDIMIENTO_ACADÉMICO': y_test_pred_labels
})
submission.to_csv('Chisma.csv', index=False)

print(f"Mejor modelo guardado: {type(best_model).__name__} con Accuracy: {best_accuracy}")
print("Predicciones guardadas en Chisma.csv")

Mejor modelo guardado: HistGradientBoostingClassifier con Accuracy: 0.5985788794591745
Predicciones guardadas en Chisma.csv
